# Importing Modules

In [1]:
# Cleanning
import pandas as pd
import numpy as np
from scipy.stats import mode

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Modelling
from scipy.stats import boxcox
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import (StandardScaler, MinMaxScaler)
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from scipy.stats import iqr
from scipy.stats import scoreatpercentile as pct


 # Units

In [54]:
sqft_to_sqm = (0.3048)**2

# Functions

## Data Cleaning Pipeline Functions

### Importing Data

In [2]:
def load_data(file):

    path = "Data/" + file

    file1 = pd.read_excel(path)

    return file1

### Standardizing Headings

In [8]:
def standard_headings(df):
    """
    Returns a Pandas Dataframe with an standarized heading, i.e lower case and " " replaced by "_"
    
    Parameters
    ----------
    columns_list : Pandas Dataframe

    Returns
    -------
    Returns a Pandas Dataframe with an standarized heading, i.e lower case and " " replaced by "_"

    """  
    heading = df.columns
    df.columns = [clabel.lower().replace(" ", "_") for clabel in heading]
    return df


## Utility Functions

In [61]:
# Function to store in dictionary the number of nan values per column

def nan_counter(df):
    
    """
    Returns a dictionary containing the number of nan values per column (for dataframe df)

    Parameters
    ----------
    df : Pandas dataframe

    Returns
    -------
    remaining_nan : Dictionary
        Contains the number of nan values in each column of the dataframe

    """
    
    remaining_nan = {}

    for column in df.columns:

        remaining_nan[column] = df[column][df[column].isna() == True].size

    return remaining_nan

# Pipeline Controller

In [9]:
# Inputs for the Pipeline Controller

drop_columns = ["id"]
# bucket_columns = ["st", "gender"]
# clean_columns_NaN = ["customer_lifetime_value", "income", "monthly_premium_auto", "number_of_open_complaints", "total_claim_amount"]
# clean_columns_zeros = ["income"]
# clean_columns_cat = [("gender", "U")]
# float_columns = ["customer_lifetime_value", "income", "monthly_premium_auto", "number_of_open_complaints", "total_claim_amount"]
# obj_columns = ["st", "gender", "education", "number_of_open_complaints", "policy_type", "vehicle_class"]

# bucketing_dict = {"F":"F", "female":"F", "Femal":"F"
#                  ,"M":"M", "Male":"M"
#                  ,"California": "California", "Cali": "California"
#                  ,"Arizona":"Arizona","AZ":"Arizona"
#                  ,"Washington":"Washington", "WA":"Washington"
#                  ,"Oregon":"Oregon"
#                  ,"Nevada":"Nevada"
#                  ,np.nan:np.nan
#                  }

In [10]:
hp_df = (load_data("Data_MidTerm_Project_Real_State_Regression.xls")
.pipe(standard_headings)
.drop(drop_columns, axis=1)
# .pipe(dt_corrector)
# .pipe(bucket_series, bucket_columns, bucketing_dict)
# .drop_duplicates().reset_index(drop=True)
# .pipe(replace_by, clean_columns_NaN)
# .pipe(replace_by, clean_columns_zeros)
# .pipe(replace_by_cat, clean_columns_cat)
# .dropna()
# .pipe(standardize_data)
# .pipe(float_to_int, float_columns)
# .pipe(obj_to_cat, obj_columns)
)

# Initial Data Exploration

In [11]:
hp_df

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,2014-10-13,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,221900
1,2014-12-09,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,538000
2,2015-02-25,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,180000
3,2014-12-09,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,604000
4,2015-02-18,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,510000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,2014-05-21,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509,360000
21593,2015-02-23,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200,400000
21594,2014-06-23,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007,402101
21595,2015-01-16,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287,400000


In [60]:
hp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           21597 non-null  datetime64[ns]
 1   bedrooms       21597 non-null  int64         
 2   bathrooms      21597 non-null  float64       
 3   sqft_living    21597 non-null  int64         
 4   sqft_lot       21597 non-null  int64         
 5   floors         21597 non-null  float64       
 6   waterfront     21597 non-null  int64         
 7   view           21597 non-null  int64         
 8   condition      21597 non-null  int64         
 9   grade          21597 non-null  int64         
 10  sqft_above     21597 non-null  int64         
 11  sqft_basement  21597 non-null  int64         
 12  yr_built       21597 non-null  int64         
 13  yr_renovated   21597 non-null  int64         
 14  zipcode        21597 non-null  int64         
 15  lat            2159

In [67]:
hp_df_shape = hp_df.shape
hp_df_shape

(21597, 20)

In [62]:
nan_counter(hp_df)

{'date': 0,
 'bedrooms': 0,
 'bathrooms': 0,
 'sqft_living': 0,
 'sqft_lot': 0,
 'floors': 0,
 'waterfront': 0,
 'view': 0,
 'condition': 0,
 'grade': 0,
 'sqft_above': 0,
 'sqft_basement': 0,
 'yr_built': 0,
 'yr_renovated': 0,
 'zipcode': 0,
 'lat': 0,
 'long': 0,
 'sqft_living15': 0,
 'sqft_lot15': 0,
 'price': 0}

In [93]:
hp_df.drop_duplicates().shape

(21597, 20)

In [90]:
# Records with yr_renovated = 0

yr_renovated_n_0 = hp_df.yr_renovated.value_counts(dropna=False).sort_index()

print(f'The number of record with availabe info regarding the renovation year is {hp_df_shape[0] - yr_renovated_n_0[0]}')


The number of record with availabe info regarding the renovation year is 914


In [91]:
# Houses where the living area in 2015 is different to that when built. This is a strong indication that a renovation has been carried out.

renovated_houses = hp_df[hp_df["sqft_living"] != hp_df["sqft_living15"]]
print(f'The number of record that corresponde to renovated houses is {renovated_houses.shape[0]}')

The number of record that corresponde to renovated houses is 19034


In [75]:
# Houses where the living area in 2015 is equak to that when built.

hp_df[(hp_df["sqft_living"] == hp_df["sqft_living15"]) & hp_df["yr_renovated"] == 0]

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,2014-10-13,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,221900
1,2014-12-09,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,538000
2,2015-02-25,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,180000
3,2014-12-09,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,604000
4,2015-02-18,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,510000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,2014-05-21,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509,360000
21593,2015-02-23,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200,400000
21594,2014-06-23,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007,402101
21595,2015-01-16,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287,400000


In [81]:
hp_df[(hp_df["grade"] < 6) & (hp_df["yr_renovated"]!=0)]

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
834,2014-11-20,3,1.75,1380,14000,1.0,0,0,4,5,1380,0,1939,1957,98001,47.2940,-122.281,1490,18503,249900
8157,2014-06-19,3,2.00,1340,4320,1.0,0,0,3,5,920,420,1912,1993,98002,47.2990,-122.228,980,6480,186000
9373,2014-10-24,3,1.75,1420,3000,1.0,0,0,3,5,710,710,1931,2014,98178,47.4928,-122.274,1960,5000,230000
9632,2014-08-14,2,0.75,520,6862,1.0,0,0,4,4,520,0,1924,1980,98010,47.3260,-122.037,1170,8756,330000
12811,2015-01-16,1,1.00,680,21727,1.0,0,0,3,5,680,0,1952,1995,98058,47.4460,-122.175,1470,19406,291970
12823,2014-10-13,1,1.00,900,7500,1.0,0,0,3,5,900,0,1946,1987,98155,47.7553,-122.283,1470,7500,305000
15526,2015-05-12,2,1.00,600,6120,1.0,0,0,3,5,600,0,1943,1989,98168,47.5000,-122.317,1090,6120,135000
